In [93]:
import pandas as pd
import numpy as np

ESサイトのユーザーに対して**RCT**を適用したメールマーケティングを行ったデータ。

In [94]:
# データのダウンロード
mail = pd.read_csv('http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')

In [95]:
print(mail.shape)
mail.head()

(64000, 12)


,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


In [96]:
print(len(mail.query("segment == 'No E-Mail'")))
print(len(mail.query("segment == 'Womens E-Mail'")))
print(len(mail.query("segment == 'Mens E-Mail'")))

21306
21387
21307


## RCTのデータからATEを求めてBaseLineとする
簡略化のために女性向けのメールが配信されているデータを削除する。

In [97]:
mail_df = mail.query("segment != 'Womens E-Mail'")
print(len(mail.query("segment == 'No E-Mail'")))
print(len(mail.query("segment == 'Womens E-Mail'")))
print(len(mail.query("segment == 'Mens E-Mail'")))

21306
21387
21307


In [98]:
mail_df["segment"] = mail_df.segment.map(lambda x: 1 if x == 'Mens E-Mail' else 0)
mail_df = mail_df.rename(columns={"segment" : "treatment"})
mail_df.head()

/Users/ryoto/opt/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,recency,history_segment,history,mens,womens,zip_code,newbie,channel,treatment,visit,conversion,spend
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,0,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,1,0,0,0.0
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,1,0,0,0.0
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,1,1,0,0.0
14,4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,0,0,0,0.0


In [99]:
mail_df.groupby("treatment").agg({"spend" : "mean", "conversion": "mean", "visit":"count"}).rename(columns={"visit" : "count"})

,spend,conversion,count
treatment,,,
0,0.652789,0.005726,21306
1,1.422617,0.012531,21307


In [100]:
# treatment 0, 1 のデータフレームに分ける
treatment_1 = mail_df.query("treatment == 1")
treatment_0 = mail_df.query("treatment == 0")

In [101]:
# 介入が購買金額に与えた影響を計算する
ts_1 = treatment_1["spend"].mean()
ts_0 = treatment_0["spend"].mean()
print("介入が購買金額に与えた影響は,", np.round(ts_1 - ts_0, 3))

介入が購買金額に与えた影響は, 0.77


In [102]:
# 介入がconversionに与えた影響を計算する
tc_1 = treatment_1["conversion"].mean()
tc_0 = treatment_0["conversion"].mean()
print("介入がconversionに与えた影響は,", np.round((tc_1 - tc_0)*100, 3), "%")

介入がconversionに与えた影響は, 0.681 %


## Meta-LearnerでDemoをやってみる

In [103]:
mail_df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,treatment,visit,conversion,spend
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,0,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,1,0,0,0.0
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,1,0,0,0.0
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,1,1,0,0.0
14,4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,0,0,0,0.0


In [104]:
# ラベルエンコーディング（OrdinalEncoder）
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder()
encoded = oe.fit_transform(mail_df[['history_segment', 'zip_code', "channel"]].values)
# decoded = oe.inverse_transform(encoded)

print('エンコード結果: ')
encoded_df = pd.DataFrame(encoded, columns = ["history_segment", "zip_code", "channel"])
encoded_df.head()

エンコード結果: 


,history_segment,zip_code,channel
0,2.0,0.0,2.0
1,4.0,0.0,2.0
2,4.0,0.0,1.0
3,1.0,2.0,2.0
4,2.0,0.0,0.0


In [105]:
mail_df_use = mail_df[
    ['recency', 'history', 'mens', 'womens', 'newbie', 'treatment', 'visit', 'conversion', 'spend']
    ]
demo_df = pd.concat([mail_df_use.reset_index(), encoded_df], axis = 1)
demo_df.head()

,index,recency,history,mens,womens,newbie,treatment,visit,conversion,spend,history_segment,zip_code,channel
0,1,6,329.08,1,1,1,0,0,0,0.0,2.0,0.0,2.0
1,3,9,675.83,1,0,1,1,0,0,0.0,4.0,0.0,2.0
2,8,9,675.07,1,1,1,1,0,0,0.0,4.0,0.0,1.0
3,13,2,101.64,0,1,0,1,1,0,0.0,1.0,2.0,2.0
4,14,4,241.42,0,1,1,0,0,0,0.0,2.0,0.0,0.0


In [110]:
# 集団を2つに分ける
df_0 = demo_df.query("treatment == 0")  # 介入を受けていない集団
df_1 = demo_df.query("treatment == 1")  # 介入を受けた集団

X_0 = df_0.drop(columns=["treatment", "spend"])
X_1 = df_1.drop(columns=["treatment", "spend"])
y_0 = df_0[["spend"]].astype(int)
y_1 = df_1[["spend"]].astype(int)

In [111]:
# ランダムフォレストモデルを作成
from sklearn.ensemble import GradientBoostingClassifier

# 介入を受けていないモデル
reg_0 =  GradientBoostingClassifier(n_estimators=100, max_depth=6, min_samples_leaf=int(10))
reg_0.fit(X_0, y_0)

# 介入を受けたモデル
reg_1 = models = GradientBoostingClassifier(n_estimators=100, max_depth=6, min_samples_leaf=int(10))
reg_1.fit(X_1, y_1)

/Users/ryoto/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [ ]:
# ATEを求める
X = demo_df.drop(columns = ["treatment","spend"]) 
mu_0 = reg_0.predict(X)
mu_1 = reg_1.predict(X)

ATE = (mu_1 - mu_0).mean()
print("ATE：", ATE)

ATE： -0.037574075139350106
